In [1]:
import os
import time
import copy
import random
import numpy as np
from skimage.color import gray2rgb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models
from torchsummary import summary

from config import models_folder, output_data_folder
from config import n_mels

from model_definitions import SpectrogramEncoderNet, MultiSiameseContrastiveClassifierNet
from project_utils import ModelSaveAndLogHandler

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
IMG_HEIGHT = n_mels
CANDIDATE_SIZE = 3

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.cuda.is_available()

cuda


True

In [5]:
# # Mobile net
# model = models.mobilenet_v2(pretrained=False)
# summary(model, input_size=(3, IMG_HEIGHT, IMG_HEIGHT), device='cpu')

In [6]:
## mobile net classifier
# model.classifier

In [7]:
summary(SpectrogramEncoderNet(), input_size=(3, IMG_HEIGHT, IMG_HEIGHT), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             864
       BatchNorm2d-2           [-1, 32, 64, 64]              64
             ReLU6-3           [-1, 32, 64, 64]               0
            Conv2d-4           [-1, 32, 64, 64]             288
       BatchNorm2d-5           [-1, 32, 64, 64]              64
             ReLU6-6           [-1, 32, 64, 64]               0
            Conv2d-7           [-1, 16, 64, 64]             512
       BatchNorm2d-8           [-1, 16, 64, 64]              32
  InvertedResidual-9           [-1, 16, 64, 64]               0
           Conv2d-10           [-1, 96, 64, 64]           1,536
      BatchNorm2d-11           [-1, 96, 64, 64]             192
            ReLU6-12           [-1, 96, 64, 64]               0
           Conv2d-13           [-1, 96, 32, 32]             864
      BatchNorm2d-14           [-1, 96,

     BatchNorm2d-125            [-1, 160, 4, 4]             320
InvertedResidual-126            [-1, 160, 4, 4]               0
          Conv2d-127            [-1, 960, 4, 4]         153,600
     BatchNorm2d-128            [-1, 960, 4, 4]           1,920
           ReLU6-129            [-1, 960, 4, 4]               0
          Conv2d-130            [-1, 960, 4, 4]           8,640
     BatchNorm2d-131            [-1, 960, 4, 4]           1,920
           ReLU6-132            [-1, 960, 4, 4]               0
          Conv2d-133            [-1, 160, 4, 4]         153,600
     BatchNorm2d-134            [-1, 160, 4, 4]             320
InvertedResidual-135            [-1, 160, 4, 4]               0
          Conv2d-136            [-1, 960, 4, 4]         153,600
     BatchNorm2d-137            [-1, 960, 4, 4]           1,920
           ReLU6-138            [-1, 960, 4, 4]               0
          Conv2d-139            [-1, 960, 4, 4]           8,640
     BatchNorm2d-140            [-1, 960

In [8]:
summary(MultiSiameseContrastiveClassifierNet(), input_size=(CANDIDATE_SIZE+1, 3, IMG_HEIGHT, IMG_HEIGHT), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             864
       BatchNorm2d-2           [-1, 32, 64, 64]              64
             ReLU6-3           [-1, 32, 64, 64]               0
            Conv2d-4           [-1, 32, 64, 64]             288
       BatchNorm2d-5           [-1, 32, 64, 64]              64
             ReLU6-6           [-1, 32, 64, 64]               0
            Conv2d-7           [-1, 16, 64, 64]             512
       BatchNorm2d-8           [-1, 16, 64, 64]              32
  InvertedResidual-9           [-1, 16, 64, 64]               0
           Conv2d-10           [-1, 96, 64, 64]           1,536
      BatchNorm2d-11           [-1, 96, 64, 64]             192
            ReLU6-12           [-1, 96, 64, 64]               0
           Conv2d-13           [-1, 96, 32, 32]             864
      BatchNorm2d-14           [-1, 96,

     BatchNorm2d-125            [-1, 160, 4, 4]             320
InvertedResidual-126            [-1, 160, 4, 4]               0
          Conv2d-127            [-1, 960, 4, 4]         153,600
     BatchNorm2d-128            [-1, 960, 4, 4]           1,920
           ReLU6-129            [-1, 960, 4, 4]               0
          Conv2d-130            [-1, 960, 4, 4]           8,640
     BatchNorm2d-131            [-1, 960, 4, 4]           1,920
           ReLU6-132            [-1, 960, 4, 4]               0
          Conv2d-133            [-1, 160, 4, 4]         153,600
     BatchNorm2d-134            [-1, 160, 4, 4]             320
InvertedResidual-135            [-1, 160, 4, 4]               0
          Conv2d-136            [-1, 960, 4, 4]         153,600
     BatchNorm2d-137            [-1, 960, 4, 4]           1,920
           ReLU6-138            [-1, 960, 4, 4]               0
          Conv2d-139            [-1, 960, 4, 4]           8,640
     BatchNorm2d-140            [-1, 960

     BatchNorm2d-252            [-1, 384, 8, 8]             768
           ReLU6-253            [-1, 384, 8, 8]               0
          Conv2d-254            [-1, 384, 8, 8]           3,456
     BatchNorm2d-255            [-1, 384, 8, 8]             768
           ReLU6-256            [-1, 384, 8, 8]               0
          Conv2d-257             [-1, 96, 8, 8]          36,864
     BatchNorm2d-258             [-1, 96, 8, 8]             192
InvertedResidual-259             [-1, 96, 8, 8]               0
          Conv2d-260            [-1, 576, 8, 8]          55,296
     BatchNorm2d-261            [-1, 576, 8, 8]           1,152
           ReLU6-262            [-1, 576, 8, 8]               0
          Conv2d-263            [-1, 576, 8, 8]           5,184
     BatchNorm2d-264            [-1, 576, 8, 8]           1,152
           ReLU6-265            [-1, 576, 8, 8]               0
          Conv2d-266             [-1, 96, 8, 8]          55,296
     BatchNorm2d-267             [-1, 96

In [9]:
class DataGenerator:
    
    def __init__(self, spectrogram_samples_files, candidate_size, batch_size, num_batches, num_sub_samples, img_height):
        self.spectrogram_samples_files = spectrogram_samples_files   # list of filepaths
        self.candidate_size = candidate_size   # 1 positive, n-1 negatives
        self.batch_size = batch_size   # batch size
        self.num_batches = num_batches   # num batches per epoch
        self.num_sub_samples = num_sub_samples   # num sub-samples per epoch
        self.img_height = img_height   # height of square img to be generated in the batches
        self.sub_samples = []   # list of RGB converted spectrograms
    
    def generate_batches(self):
        while True:
            self.create_sub_samples()
            # batches per epoch
            for _ in range(self.num_batches):
                # create batch
                labels = []
                query_and_candidate_imgs = [[] for _ in range(self.candidate_size + 1)]
                for _ in range(self.batch_size):
                    sample_spectrograms_indices = random.sample(range(self.num_sub_samples), self.candidate_size)   # sample candidates
                    pos_idx = sample_spectrograms_indices[0]   # positive sample
                    # Generate query image
                    query_img = self.get_sliding_img_slice_from_spectrogram(self.sub_samples[pos_idx])
                    # Generate batch images
                    random.shuffle(sample_spectrograms_indices)
                    candidate_imgs = [self.get_sliding_img_slice_from_spectrogram(self.sub_samples[idx]) for idx in sample_spectrograms_indices]
                    # get class label / idx of positive sample
                    pos_candidate_idx = sample_spectrograms_indices.index(pos_idx)   
                    labels.append(pos_candidate_idx)
                    # Normalize input imgs
                    for i, img in enumerate([query_img, *candidate_imgs]):
                        img = img / np.amax(np.absolute(img))   # normalize to range [-1, 1]
                        query_and_candidate_imgs[i].append(img)
                # Convert to tensor
                labels = torch.tensor(labels)
                input_imgs = torch.tensor(query_and_candidate_imgs)
                yield (input_imgs, labels)
    
    def create_sub_samples(self):
        self.sub_samples = []   # reset
        files = random.sample(self.spectrogram_samples_files, self.num_sub_samples)   # sampling without replacement
        for file in files:
#             print(file)
            spectrogram = np.load(file)
            assert spectrogram.shape[0] == self.img_height, "Input spectrogram height does not match img height"
            self.sub_samples.append(spectrogram)
    
    @classmethod
    def get_sliding_img_slice_from_spectrogram(cls, spectrogram, depth=3, sliding_ratio=2):
        ### Combine multiple sliding greyscale img slices into an n-depth image
        height = spectrogram.shape[0]
        slide_step = height//sliding_ratio
        img_slice = np.zeros((depth,height,height))   # initialize empty img (pytorch style)
        # Get random start idx
        slice_start = random.randint(0, spectrogram.shape[1] - (slide_step*(depth+1)) - 1)
        for i in range(depth):
            img_slice[i,:,:] = spectrogram[:, slice_start:slice_start+height]   # get slice (pytorch style)
            slice_start += slide_step   # slide
        img_slice = img_slice.astype("float32")
        return img_slice
    
    @classmethod
    def spectrogram_to_RGB(cls, spectrogram):
        assert len(spectrogram.shape) == 2, "Spectrogram input should be a 2D array"
        spectrogram_rgb = gray2rgb(spectrogram)
        return spectrogram_rgb
        

In [10]:
training_folder = os.path.join(output_data_folder, "training_dataset_full_spectrogram/vox1_dev_wav")
spectrogram_samples_files = [os.path.join(training_folder, file) for file in os.listdir(training_folder)]
candidate_size = CANDIDATE_SIZE
batch_size = 25
num_batches = 2000 // batch_size
# num_batches = 200 // batch_size
num_sub_samples = 200
# num_sub_samples = 20
training_data_generator = DataGenerator(spectrogram_samples_files, candidate_size, batch_size, num_batches, num_sub_samples, IMG_HEIGHT)

In [11]:
def train_model(model, criterion, optimizer, scheduler, num_epochs, num_batches, training_data_generator, log_handler):
    since = time.time()

#     best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    t1 = time.time()
    for epoch in range(num_epochs):
        log_handler.print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        log_handler.print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            num_batch_modified = num_batches if phase == 'train' else (num_batches // 5) + 1
            for i, data in zip(range(num_batch_modified), training_data_generator.generate_batches()):               
                input_imgs, labels = data
                inputs = [img.to(device) for img in input_imgs]
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):   # gradient only for train
                    outputs = model(inputs)   
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs[0].size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / (num_batch_modified * inputs[0].size(0))
            epoch_acc = running_corrects.double() / (num_batch_modified * inputs[0].size(0))

            log_handler.print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())
                log_handler.save_pytorch_model(model, "best_model_{}.pt".format(model.__class__.__name__))

        # end of epoch
        log_handler.print("Time taken is {} seconds".format(int(time.time()-t1)))
        t1 = time.time()
        log_handler.print()

    time_elapsed = time.time() - since
    log_handler.print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    log_handler.print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
#     model.load_state_dict(best_model_wts)
#     return model

In [12]:
### Train

epochs = 50

model_ft = MultiSiameseContrastiveClassifierNet().to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr = 0.0001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)


### Train 

# Logger
model_save_folder = os.path.join(models_folder, "contrastive_encoder")
log_handler = ModelSaveAndLogHandler(model_save_folder, enable_model_saving=True, enable_logging=True)   # init
model_def_src_file_path = os.path.join(r"D:\Desktop\projects\speaker_recognition_voxceleb1\scripts", "model_definitions.py")
log_handler.save_model_definition_file(model_def_src_file_path)   # copy model def file
print(log_handler.folder)

# Description
log_handler.print("Description: Candidates: 3, Encoding: 128, Projection: None")

# Train
train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, epochs, num_batches, training_data_generator, log_handler)


D:\Desktop\projects\speaker_recognition_voxceleb1\output_data\models\contrastive_encoder\2020-03-18_02-09-45
Description: Candidates: 3, Encoding: 128, Projection: None
Epoch 0/49
----------
train Loss: 1.0478 Acc: 0.4345
val Loss: 1.0125 Acc: 0.4682
MODEL SAVED
Time taken is 105 seconds

Epoch 1/49
----------
train Loss: 1.0325 Acc: 0.4505
val Loss: 1.0229 Acc: 0.4612
Time taken is 108 seconds

Epoch 2/49
----------
train Loss: 1.0457 Acc: 0.4470
val Loss: 1.0336 Acc: 0.4329
Time taken is 96 seconds

Epoch 3/49
----------
train Loss: 1.0252 Acc: 0.4585
val Loss: 1.0205 Acc: 0.4494
Time taken is 109 seconds

Epoch 4/49
----------
train Loss: 1.0158 Acc: 0.4810
val Loss: 0.9932 Acc: 0.4824
MODEL SAVED
Time taken is 106 seconds

Epoch 5/49
----------
train Loss: 1.0002 Acc: 0.4675
val Loss: 0.9954 Acc: 0.4871
MODEL SAVED
Time taken is 99 seconds

Epoch 6/49
----------
train Loss: 1.0086 Acc: 0.4750
val Loss: 1.0012 Acc: 0.4306
Time taken is 104 seconds

Epoch 7/49
----------
train Loss: 

In [13]:
random_acc = 1 / CANDIDATE_SIZE
random_acc

0.3333333333333333

### TODO: Overall
* Contrastive classifier
    * separate train and validate methods

* (Done) Model saving / checkpointing
* **Build binary classifier**